In [1]:
# Requirements
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Load Data & Obtain Graphs

In [2]:
# Load DataFrame (self-loops removed)
trans_3w = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\trans_3w_cl.csv')
trans_3w = trans_3w[trans_3w.ammount > 0]
trans_3w.head()

,txn_hash,input_address,output_address,ammount,fees,block_index,block_time,input_flag,output_flag
0,bd36f2ca16e2a2c73c807b7d1569657b30de8453450cd2...,13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR,16FPyvvz5Ug3cx97qH67KfgC6PY1S9fskQ,2.420000e+07,320000.000000,453318,2017-02-16 12:05:04,0,1
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2.503648e+03,3.314845,453318,2017-02-16 12:05:04,0,1
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,6.359402e+05,841.988605,453318,2017-02-16 12:05:04,0,1
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,5.902369e+05,781.477250,453318,2017-02-16 12:05:04,0,1
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc...,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,1.013032e+05,134.126076,453318,2017-02-16 12:05:04,0,1


Slef-loops, i.e. rows where the input and output addresses are the same, are excluded because these reflect the remaining balance in an address's crypto walet or the transfer of change. If the transfer of change goes to the same account this is taken as an indication that a user has nothing to hide, but if a user transfers any change received from a transaction to another address this is considered as being an attempt to disperse funds (reduce transparency).

Keeping self-loops also has implications for sampling methods. In the case of node sampling methods, self-loops can result in a sample meeting the edge condition (seen in samplers below) whilst still having several isolated nodes. In the case of random walks, self-loops can cause the walk to circle back and avoid nodes that would otherwise be sampled.

In [3]:
# Obtain full network NetworkX Graph
G_full = nx.from_pandas_edgelist(trans_3w, 'input_address', 'output_address', edge_attr=['txn_hash', 'ammount', 'fees', 
                                'block_index', 'block_time'], create_using=nx.MultiDiGraph())

# Confirm that Graph is Directed
nx.is_directed(G_full)

True

In [4]:
# Generator for giant component
giant = max(nx.connected_components(G_full.to_undirected()))

# Get sub-graph of Giant Component
G_giant = G_full.subgraph(giant)
G_giant = G_giant.to_directed()

print(nx.info(G_giant))

MultiDiGraph with 3827352 nodes and 11724249 edges


In [5]:
# Remove unnecassary objects from memory
del trans_3w
del G_full

## Random Node Sampling

When running the RandomNodeSampler it was noted that the method took an extremely long time to find a representative sample due to 2 limitations: 1. a large sample, and 2. a weakly connected component that makes it difficult to satify the edge condition. Hence, the sample of nodes is adjusted to exclude nodes that would slow down this process. 

This sub-sampling is aimed at ensuring that all fraudulent/high-risk nodes are included, and that low centrality nodes are excluded such that they don't slow down the process. 

### Random Node Sampler

In [6]:
# List all nodes in giant component
n_giant = G_giant.nodes

In [7]:
# Get node properties from data exploration exercise
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')

# Describe properties overall
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [8]:
# Subset node_properties by addresses found in giant component
node_properties = node_properties[node_properties['address'].isin(list(n_giant))]
node_properties.shape

(3827352, 6)

In [9]:
# Subset by fraudulent/high-risk nodes

fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()

In [10]:
# Remove nodes with low centrality

licit_nodes = node_properties[node_properties['fraud_flag'] == 0] # sample from low-risk nodes
licit_nodes = licit_nodes[licit_nodes['eigen_centrality'] > 1.818528e-6] # remove nodes in bottom 84% of centrality
licit_nodes = licit_nodes['address'].tolist()
len(licit_nodes)

362814

*Adjusted methods to account for pre-randomisation sub-sampling approach*

In [11]:
class RandomNodeSampler:
    def __init__(self, graph, fraud_sample, licit_sample, number_of_nodes):
        self.number_of_nodes = number_of_nodes
        self.fraud_sample = fraud_sample
        self.licit_sample = licit_sample 
        
    def sampler(self, graph, fraud_sample, licit_sample, number_of_nodes):
        s_size = self.number_of_nodes - len(fraud_sample) # Number of licit nodes to sample
        rd_nodes = rd.sample(self.licit_sample, s_size) # Randomly select licit of nodes
        rd_nodes = rd_nodes + self.fraud_sample  # Get full list of nodes
        g_sampler = graph.subgraph(rd_nodes) # Obtain sub-graph from randomly selected nodes
        return g_sampler
        
    def sample(self, graph, fraud_sample, licit_sample, number_of_nodes):
        new_graph = self.sampler(graph, self.fraud_sample, self.licit_sample, self.number_of_nodes) # Get randomly sampled graph
        # Edge condition: Number of Edges >= Number of Nodes (Ensures graph is not too sparse)
        while new_graph.number_of_edges() < self.number_of_nodes: # If condition not met
            new_graph = self.sampler(graph, self.fraud_sample, self.licit_sample, self.number_of_nodes) # Get new sub-graph
            if new_graph.number_of_edges() >= self.number_of_nodes:
                        break
        return new_graph

In [40]:
# Instantiate Sampler
rns_generator = RandomNodeSampler(G_giant, fr_nodes, licit_nodes, 50000)

# Obtain sub-sample
G_rns = rns_generator.sample(G_giant, fr_nodes, licit_nodes, 50000)

# Check if directed structure is maintained
nx.is_directed(G_rns)

True

In [41]:
print(nx.info(G_rns))

MultiDiGraph with 50000 nodes and 119883 edges


##### Check Properties of Random Sample

*DataFrame*

In [42]:
# Get DataFrame
rns_df = nx.to_pandas_edgelist(G_rns) 
print(rns_df.shape)

# Save Graph Data
rns_df['timestamp'] = (rns_df['block_index'] - rns_df['block_index'].min() + 1).astype(int)
new_var_names = {'ammount': 'weight'}
rns_df.rename(new_var_names, axis=1, inplace=True)
rns_df = rns_df[['source', 'target', 'weight', 'timestamp']]
rns_df.to_csv("g_ss_rn_9.csv", index = False)
#nx.write_gml(G_rns, "G_rns_multi.gml")

rns_df.head()

(119883, 7)


,source,target,weight,timestamp
0,1DLyww6d47BNnddWU4wXhQSUBSfTh7FRiS,13hB6VxWn52YuzXJJ9FE6Y7NVnT5CtRjmD,6.196600e+04,1436
1,1DLyww6d47BNnddWU4wXhQSUBSfTh7FRiS,37p9pUugydmoLpQyFLLqGAgjWmUFERa1Pq,6.331221e+06,1436
2,15pLhEpvChTTzj4wisKPq4oxCp2FYNQQWP,1E7jvxutUZHr9Ktb77MQnnhSp4QBRbqR1n,3.583969e+06,3016
3,3Fvmy3c5kjpcj86KpNEmFh2UwcaEhixWf1,1MkQ4FASsFtbyG2LU38srYr3jJ8yDPcLhH,3.488078e+04,2236
4,3Fvmy3c5kjpcj86KpNEmFh2UwcaEhixWf1,1BmQouB1EmAs1dqkjwhAQZDdduoQu2bA9d,2.107825e+05,2236


In [15]:
# Number of unique transactions
rns_df['txn_hash'].nunique()

49791

In [16]:
# Number of unique input addresses
rns_df['source'].nunique()

9914

In [17]:
# Number of unique output addresses
rns_df['target'].nunique()

25500

This is considered to be an adequate level of variety of transactions and addresses to be representative. Hovever, the difference in unique input and output addresses is not reflective of the population.

In [18]:
# View transaction properties
rns_df.describe()

,fees,ammount,block_index
count,1.194460e+05,1.194460e+05,119446.000000
mean,8.222181e+03,2.017728e+07,454410.586441
std,3.637283e+04,2.008483e+08,917.605971
min,0.000000e+00,-2.442296e+09,453318.000000
25%,2.969728e+01,1.029526e+04,453536.000000
50%,2.944913e+02,1.367488e+05,454254.000000
75%,4.836604e+03,1.573000e+06,455165.000000
max,4.080994e+06,4.793483e+10,456437.000000


A distribution skewed towards large amounts is maintained. We also note that the presence of negative values is also kept.

In [19]:
# Number of unique timestamps
rns_df['block_time'].nunique() # Near all timestamps are represented

1516

*Nodes*

In [20]:
# Get Properties

address = [node for (node, val) in G_rns.degree()]
degree = [val for (node, val) in G_rns.degree()]
in_degree = [val for (node, val) in G_rns.in_degree()]
out_degree = [val for (node, val) in G_rns.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rns) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [21]:
# Add Properties to DataFrame

nodes_rns = pd.DataFrame()
nodes_rns['address'] = address
nodes_rns['degree'] = degree
nodes_rns['in_degree'] = in_degree
nodes_rns['out_degree'] = out_degree
#nodes_rns['eigen_centrality'] = eigen_centrality

In [22]:
# Add fraud flag

nodes_rns = nodes_rns.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes_rns.loc[nodes_rns.address == i, 'fraud_flag'] = 1
    
nodes_rns.head()

,address,degree,in_degree,out_degree,fraud_flag
0,34LxvX15nrj1TW9pnsPC1uLPewoEb6dYrF,50,22,28,1
1,3PZTjWQB5je9DqZevMsHSGKxzFGCXjQreE,1,1,0,0
2,19A6r9cTmNjYewpU5f7xGBUGwGhEEujXJ5,12,3,9,0
3,3DaSHyQSVGrdSZXPNhqxRKVEjDMxQ1DHKh,5,2,3,0
4,1KwtDLNmF5gpJo2hJt4L3mWmRkcKYdnkGR,0,0,0,0


In [23]:
# Describe sample properties overall
nodes_rns[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.000000,50000.000000,50000.000000
mean,4.777840,2.388920,2.388920
std,95.123873,31.656664,74.113177
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000
75%,2.000000,1.000000,0.000000
max,11985.000000,3089.000000,9574.000000


In [24]:
# Describe Properties of Fraudulent Nodes

nodes_rns_fraud = nodes_rns[nodes_rns['fraud_flag'] == 1]
nodes_rns_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,4400.000000,4400.000000,4400.000000
mean,28.391364,9.440909,18.950455
std,317.890470,102.996720,248.606483
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,3.000000,1.000000,1.000000
max,11985.000000,3089.000000,9574.000000


In [25]:
# Describe Properties of Non-Fraudulent Nodes

nodes_rns_licit = nodes_rns[nodes_rns['fraud_flag'] == 0]
nodes_rns_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,45600.000000,45600.000000,45600.000000
mean,2.499342,1.708465,0.790877
std,10.668828,8.377521,5.596814
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000
75%,2.000000,1.000000,0.000000
max,1242.000000,1221.000000,476.000000


At the mean level the overall properties appear reflective but the presence of isolated nodes (no degrees) is a potential concern. Furthermore, the distribution of out degrees does not appear to be reflective of the giant component population.

*Graph*

In [26]:
# Density
nx.density(G_rns) # Connectivity remains very low

4.777935558711174e-05

In [27]:
# Communities - k-Cliques

com_generator_rns = k_clique_communities(G_rns.to_undirected(), k=9) # Doesn't work for k>9
com_rns = next(com_generator_rns)
print('The number of communities in randomly sampled graph is : ' + str(len(com_rns))) 

The number of communities in randomly sampled graph is : 14


This indicates that some variety in community structures was maintained. (Remember that the giant component had only 107 k-cliques).

### Degree Biased Random Sampler

In [43]:
# Get Licit nodes

licit_nodes = node_properties[node_properties['fraud_flag'] == 0] # sample from low-risk nodes
licit_nodes = licit_nodes['address'].tolist()
len(licit_nodes)

3822964

In [44]:
# Sub-sample size 
len(fr_nodes) + len(licit_nodes)

3827352

In [45]:
# Obtain sub-graph with short-listed nodes

G_licit = G_giant.subgraph(licit_nodes)

In [46]:
# Obtain probabilites

dp = node_properties[node_properties['address'].isin(licit_nodes)]
dp_sum = dp['degree'].sum()
dp['p'] = dp['degree'] / dp_sum
p_degree = dp['p'].values

In [47]:
class DegreeBasedSampler:
    def __init__(self, graph, g_licit, fraud_sample, licit_sample, number_of_nodes, p_distribution):
        self.number_of_nodes = number_of_nodes
        self.fraud_sample = fraud_sample
        self.licit_sample = licit_sample
        self.p_distribution = p_distribution

    def sampler(self, graph, g_licit, fraud_sample, licit_sample, number_of_nodes, p_distribution):
        s_size = self.number_of_nodes - len(fraud_sample) # Number of licit nodes to sample
        
        # Sample nodes and create sub-graph
        sampled_nodes = np.random.choice(self.licit_sample, s_size, replace=False, p=self.p_distribution)
        sampled_nodes = list(sampled_nodes) + list(self.fraud_sample)
        g_sampler = graph.subgraph(sampled_nodes)
        return g_sampler
    
    def sample(self, graph, g_licit, fraud_sample, licit_sample, number_of_nodes, p_distribution):
        new_graph = self.sampler(graph, g_licit, self.fraud_sample, self.licit_sample, self.number_of_nodes,
                                 self.p_distribution) # Get randomly sampled graph
        # Edge condition: Number of Edges >= Number of Nodes (Ensures graph is not too sparse)
        while new_graph.number_of_edges() < self.number_of_nodes: # If condition not met
            new_graph = self.sampler(graph, g_licit, self.fraud_sample, self.licit_sample, self.number_of_nodes, 
                                     self.p_distribution) # Get new sub-graph
            if new_graph.number_of_edges() >= self.number_of_nodes:
                        break
        return new_graph

In [77]:
# Instantiate Sampler
dbs_generator = DegreeBasedSampler(G_giant, G_licit, fr_nodes, licit_nodes, 50000, p_degree) 

# Obtain sub-sample
G_dbs = dbs_generator.sample(G_giant, G_licit, fr_nodes, licit_nodes, 50000, p_degree) 

# Check if directed structure is maintained
nx.is_directed(G_dbs) 

True

In [78]:
print(nx.info(G_dbs))

MultiDiGraph with 50000 nodes and 318854 edges


##### Check Properties of Degree Biased Random Sample

*DataFrame*

In [79]:
# Get DataFrame
dbs_df = nx.to_pandas_edgelist(G_dbs) 
print(dbs_df.shape)

# Save Graph Data
dbs_df['timestamp'] = (dbs_df['block_index'] - dbs_df['block_index'].min() + 1).astype(int)
new_var_names = {'ammount': 'weight'}
dbs_df.rename(new_var_names, axis=1, inplace=True)
dbs_df = dbs_df[['source', 'target', 'weight', 'timestamp']]
dbs_df.to_csv("g_ss_bd_9.csv", index = False)

# Save Graph
#dbs_df.to_csv("G_dbs_multidf.csv", index = False)
#nx.write_gml(G_dbs, "G_dbs_multi.gml")

dbs_df.head()

(318854, 7)


,source,target,weight,timestamp
0,3LnZpZQQ6Nk3tdnLAjg8aTRtfXouX9Jzzs,3Dcm3y3VdLb5Mc2Qhn7KcAkhBDs1BrYpmw,7.478801e+04,447
1,3LnZpZQQ6Nk3tdnLAjg8aTRtfXouX9Jzzs,3DqN3oBsRkTWPcdB3cCrvmHYboJAzbQCy1,2.173784e+05,447
2,3LnZpZQQ6Nk3tdnLAjg8aTRtfXouX9Jzzs,3KiryEyT5iJwDdhP7RZLyfGzmPCxgvYaEg,5.495510e+06,447
3,3JZUPmxt7Hjdm7kk8ymKXXcM9oGNMAXJV1,1AhNSepcJmLwXWWWoUiaLDBKJCAt7dLoVi,2.423226e+02,978
4,3JZUPmxt7Hjdm7kk8ymKXXcM9oGNMAXJV1,3MKaTNDSnbzqqYPEF6ebGQ6f9PnNLMAoQQ,1.071417e+03,978


In [36]:
# Number of unique transactions
dbs_df['txn_hash'].nunique()

62337

In [37]:
# Number of unique input addresses
dbs_df['source'].nunique()

19622

In [38]:
# Number of unique output addresses
dbs_df['target'].nunique()

19317

This is considered to be an adequate level of variety to be representative. Note that this sample has a better balance between input and output addresses.

In [39]:
# View transaction properties
dbs_df.describe()

,fees,ammount,block_index
count,3.228400e+05,3.228400e+05,322840.000000
mean,5.603200e+03,1.695694e+07,454675.291482
std,4.122036e+04,5.471067e+08,997.989720
min,0.000000e+00,-3.021075e+09,453318.000000
25%,1.444314e+01,2.567778e+03,453643.000000
50%,1.175074e+02,5.210896e+04,454637.000000
75%,1.469182e+03,7.215286e+05,455556.000000
max,7.203339e+06,2.683366e+11,456437.000000


The distribution of transaction ammounts appears to be adequately representative.

In [40]:
# Number of unique timestamps
dbs_df['block_time'].nunique() # Nearly all timestamps are preserved

1549

*Nodes*

In [41]:
# Get Properties

address = [node for (node, val) in G_dbs.degree()]
degree = [val for (node, val) in G_dbs.degree()]
in_degree = [val for (node, val) in G_dbs.in_degree()]
out_degree = [val for (node, val) in G_dbs.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_dbs) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [42]:
# Add Properties to DataFrame

nodes_dbs = pd.DataFrame()
nodes_dbs['address'] = address
nodes_dbs['degree'] = degree
nodes_dbs['in_degree'] = in_degree
nodes_dbs['out_degree'] = out_degree
#nodes_dbs['eigen_centrality'] = eigen_centrality

In [43]:
# Add fraud flag

nodes_dbs = nodes_dbs.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes_dbs.loc[nodes_dbs.address == i, 'fraud_flag'] = 1
    
nodes_dbs.head()

,address,degree,in_degree,out_degree,fraud_flag
0,1KYBdPr6dL8AnMzgacWrsojLupct3CNx4A,0,0,0,0
1,34LxvX15nrj1TW9pnsPC1uLPewoEb6dYrF,93,54,39,1
2,1LKXkHfFYjcumsq2UL5fNZXkBinS6SbzFP,0,0,0,0
3,1GfvXSsDRNCbkhYzme6xRJ8YDbk2WReSm2,1,0,1,0
4,1BbjXMhnQtFbvaBMEjuX9Npnk7R1HSVqAW,3,1,2,0


In [44]:
# Describe sample properties overall
nodes_dbs[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49999.000000,49999.000000,49999.000000
mean,12.913858,6.456929,6.456929
std,114.697504,56.932095,79.298423
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,7.000000,2.000000,2.000000
max,9643.000000,3721.000000,6799.000000


In [45]:
# Describe Properties of Fraudulent Nodes

nodes_dbs_fraud = nodes_dbs[nodes_dbs['fraud_flag'] == 1]
nodes_dbs_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,4400.000000,4400.000000,4400.000000
mean,31.875682,14.293864,17.581818
std,331.904411,150.594424,224.841481
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,5.000000,2.000000,2.000000
max,9643.000000,3721.000000,6799.000000


In [46]:
# Describe Properties of Non-Fraudulent Nodes

nodes_dbs_licit = nodes_dbs[nodes_dbs['fraud_flag'] == 0]
nodes_dbs_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,45599.000000,45599.000000,45599.000000
mean,11.084169,5.700717,5.383451
std,61.313508,36.873296,44.775423
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,7.000000,2.000000,2.000000
max,5790.000000,2306.000000,5790.000000


Degree measures are significantly skewed towards higher amounts. This bias may be mitigated with a larger sample.

*Graph*

In [47]:
# Density
nx.density(G_dbs) # connectivity remains low

0.0001291437485215963

In [48]:
# Communities - k-Cliques

com_generator_dbs = k_clique_communities(G_dbs.to_undirected(), k=10)
com_dbs = next(com_generator_dbs)
print('The number of communities in randomly sampled graph is : ' + str(len(com_dbs)))

The number of communities in randomly sampled graph is : 12


This indicates the preservation of community structures. (Note: more communities than random sampler)